In [1]:
%%writefile mapper1.py

import sys
import re
import collections

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

path = "stop_words_en.txt"

with open(path, "r") as f:
    stop_words = f.read().splitlines()
    
def cleanup(words):
    return [word.lower().strip() for word in words if (word.lower() not in stop_words)]

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        words = cleanup(words)
        
        words_counter = collections.Counter(words)
        words_total = sum(words_counter.values())

        for word, count in sorted(words_counter.items()):
            if not word.isalpha(): continue
            tf = float(count)/float(words_total)
            print("{}\t{}\t{:f}".format(word, article_id, tf))
             
    except Exception as e:
        print(e)
        continue

Writing mapper1.py


In [2]:
%%writefile reducer1.py

import sys
import math

current_word = None
article_count = 0
tf_memory = {}

for line in sys.stdin:
    try:
        word, article_id, tf = line.strip().split('\t', 2)
        tf = float(tf)
        
        if current_word != word:
            if current_word:
                idf = float(1)/math.log(1 + article_count)
                for article, tff in tf_memory.items():
                    print("{}\t{}\t{:f}".format(current_word, article, tff*idf))
            
            current_word = word
            article_count = 0
            tf_memory.clear()
        
        article_count += 1
        tf_memory[article_id] = float(tf)
    
    except Exception as e:
        print(e)
        continue
        
if current_word:
    idf = float(1)/math.log(1 + article_count)
    for word, tf in tf_memory.items():
        print("{}\t{}\t{:f}".format(current_word, article, tff*idf))

Writing reducer1.py


In [3]:
%%bash

OUT_DIR="assignment3_"$(date +"%s%6N")
NUM_REDUCERS=4

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Assignment 3" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -files mapper1.py,reducer1.py,/datasets/stop_words_en.txt \
    -mapper "python2 mapper1.py" \
    -reducer "python2 reducer1.py" \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 

hdfs dfs -cat ${OUT_DIR}/part* | grep -w "labor" | grep -w "12" | cut -f 3

bash: line 5: hdfs: command not found
bash: line 7: yarn: command not found
bash: line 18: hdfs: command not found
